In [72]:
from __future__ import division

import os
import urllib

import pymongo as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re
import ast

from PIL import Image
import base64
import sys

from IPython.display import clear_output

## plotting
import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

from collections import Counter

## svg rendering 
# import ast
# from svgpathtools import parse_path, wsvg, svg2paths

In [73]:
# directory & file hierarchy
proj_dir = os.path.abspath('../')
stimulus_dir = os.getcwd()
analysis_dir = os.path.join(proj_dir,'analysis')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','python') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis','python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
if sys.version_info[0]>=3:
    from importlib import reload

In [74]:
### set up paths, etc. 
path_to_stim = os.getcwd()

## load in run5_submitButton data
M = pd.read_csv(os.path.join(path_to_stim,'graphical_conventions_group_data_run5_submitButton.csv'))

## TODO: add several columns to dataframe above: 'target_url', 'basic', 'subset', 'cluster','width','height' column based on dictionaries here: https://github.com/cogtoolslab/semantic_mapping/issues/1
shapenet_30afd2ef2ed30238aa3d0a2f00b54836 = {'filename': "30afd2ef2ed30238aa3d0a2f00b54836.png" , 'basic': "dining", 'subordinate': "dining_00" , 'subset':"A", 'cluster': 1, 'object': 0, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/30afd2ef2ed30238aa3d0a2f00b54836.png",'width': 256, 'height': 256};
shapenet_30dc9d9cfbc01e19950c1f85d919ebc2 = {'filename': "30dc9d9cfbc01e19950c1f85d919ebc2.png" , 'basic': "dining", 'subordinate': "dining_01" , 'subset':"A", 'cluster': 1, 'object': 1, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/30dc9d9cfbc01e19950c1f85d919ebc2.png",'width': 256, 'height': 256};
shapenet_4c1777173111f2e380a88936375f2ef4 = {'filename': "4c1777173111f2e380a88936375f2ef4.png" , 'basic': "dining", 'subordinate': "dining_02" , 'subset':"B", 'cluster': 1, 'object': 2, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/4c1777173111f2e380a88936375f2ef4.png",'width': 256, 'height': 256};
shapenet_3466b6ecd040e252c215f685ba622927 = {'filename': "3466b6ecd040e252c215f685ba622927.png" , 'basic': "dining", 'subordinate': "dining_03" , 'subset':"B", 'cluster': 1, 'object': 3, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/3466b6ecd040e252c215f685ba622927.png",'width': 256, 'height': 256};
shapenet_38f87e02e850d3bd1d5ccc40b510e4bd = {'filename': "38f87e02e850d3bd1d5ccc40b510e4bd.png" , 'basic': "dining", 'subordinate': "dining_04" , 'subset':"B", 'cluster': 1, 'object': 4, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/38f87e02e850d3bd1d5ccc40b510e4bd.png",'width': 256, 'height': 256};
shapenet_3cf6db91f872d26c222659d33fd79709 = {'filename': "3cf6db91f872d26c222659d33fd79709.png" , 'basic': "dining", 'subordinate': "dining_05" , 'subset':"B", 'cluster': 1, 'object': 5, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/3cf6db91f872d26c222659d33fd79709.png",'width': 256, 'height': 256};
shapenet_3d7ebe5de86294b3f6bcd046624c43c9 = {'filename': "3d7ebe5de86294b3f6bcd046624c43c9.png" , 'basic': "dining", 'subordinate': "dining_06" , 'subset':"A", 'cluster': 1, 'object': 6, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/3d7ebe5de86294b3f6bcd046624c43c9.png",'width': 256, 'height': 256};
shapenet_56262eebe592b085d319c38340319ae4 = {'filename': "56262eebe592b085d319c38340319ae4.png" , 'basic': "dining", 'subordinate': "dining_07" , 'subset':"A", 'cluster': 1, 'object': 7, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/56262eebe592b085d319c38340319ae4.png",'width': 256, 'height': 256};
shapenet_1d1641362ad5a34ac3bd24f986301745 = {'filename': "1d1641362ad5a34ac3bd24f986301745.png" , 'basic': "waiting", 'subordinate': "waiting_00" , 'subset':"A", 'cluster': 3, 'object': 0, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/1d1641362ad5a34ac3bd24f986301745.png",'width': 256, 'height': 256};
shapenet_1da9942b2ab7082b2ba1fdc12ecb5c9e = {'filename': "1da9942b2ab7082b2ba1fdc12ecb5c9e.png" , 'basic': "waiting", 'subordinate': "waiting_01" , 'subset':"A", 'cluster': 3, 'object': 1, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/1da9942b2ab7082b2ba1fdc12ecb5c9e.png",'width': 256, 'height': 256};
shapenet_2448d9aeda5bb9b0f4b6538438a0b930 = {'filename': "2448d9aeda5bb9b0f4b6538438a0b930.png" , 'basic': "waiting", 'subordinate': "waiting_02" , 'subset':"B", 'cluster': 3, 'object': 2, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/2448d9aeda5bb9b0f4b6538438a0b930.png",'width': 256, 'height': 256};
shapenet_23b0da45f23e5fb4f4b6538438a0b930 = {'filename': "23b0da45f23e5fb4f4b6538438a0b930.png" , 'basic': "waiting", 'subordinate': "waiting_03" , 'subset':"B", 'cluster': 3, 'object': 3, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/23b0da45f23e5fb4f4b6538438a0b930.png",'width': 256, 'height': 256};
shapenet_2b5953c986dd08f2f91663a74ccd2338 = {'filename': "2b5953c986dd08f2f91663a74ccd2338.png" , 'basic': "waiting", 'subordinate': "waiting_04" , 'subset':"B", 'cluster': 3, 'object': 4, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/2b5953c986dd08f2f91663a74ccd2338.png",'width': 256, 'height': 256};
shapenet_2e291f35746e94fa62762c7262e78952 = {'filename': "2e291f35746e94fa62762c7262e78952.png" , 'basic': "waiting", 'subordinate': "waiting_05" , 'subset':"B", 'cluster': 3, 'object': 5, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/2e291f35746e94fa62762c7262e78952.png",'width': 256, 'height': 256};
shapenet_2eaab78d6e4c4f2d7b0c85d2effc7e09 = {'filename': "2eaab78d6e4c4f2d7b0c85d2effc7e09.png" , 'basic': "waiting", 'subordinate': "waiting_06" , 'subset':"A", 'cluster': 3, 'object': 6, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/2eaab78d6e4c4f2d7b0c85d2effc7e09.png",'width': 256, 'height': 256};
shapenet_309674bdec2d24d7597976c675750537 = {'filename': "309674bdec2d24d7597976c675750537.png" , 'basic': "waiting", 'subordinate': "waiting_07" , 'subset':"A", 'cluster': 3, 'object': 7, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/309674bdec2d24d7597976c675750537.png",'width': 256, 'height': 256};
## subset single example gameID and repeated object (one that we have 8 unique trialNums associated with), and 
## write this out using `.to_json()` to mock up single trial prototype
get_url = {}
l = [shapenet_30afd2ef2ed30238aa3d0a2f00b54836,\
    shapenet_30dc9d9cfbc01e19950c1f85d919ebc2,\
    shapenet_4c1777173111f2e380a88936375f2ef4,\
    shapenet_3466b6ecd040e252c215f685ba622927,\
    shapenet_38f87e02e850d3bd1d5ccc40b510e4bd,\
    shapenet_3cf6db91f872d26c222659d33fd79709,\
    shapenet_3d7ebe5de86294b3f6bcd046624c43c9,\
    shapenet_56262eebe592b085d319c38340319ae4,\
    shapenet_1d1641362ad5a34ac3bd24f986301745,\
    shapenet_1da9942b2ab7082b2ba1fdc12ecb5c9e,\
    shapenet_2448d9aeda5bb9b0f4b6538438a0b930,\
    shapenet_23b0da45f23e5fb4f4b6538438a0b930,\
    shapenet_2b5953c986dd08f2f91663a74ccd2338,\
    shapenet_2e291f35746e94fa62762c7262e78952,\
    shapenet_2eaab78d6e4c4f2d7b0c85d2effc7e09,\
    shapenet_309674bdec2d24d7597976c675750537]

for row in l:
    get_url[row['subordinate']] = row['target_url']

M['target_url'] = M.apply(lambda x: get_url[x['target']], axis = 1)

In [75]:
#Sanity Check for the data and find the ones that are repeated
 #dataframe
num_freq = {}
for i, group in M.groupby(['gameID','target']):
    num = len(set(list(group.trialNum)))
    if num not in num_freq:
        num_freq[num] = 1
    else:
        num_freq[num] +=1
    
assert list(num_freq.keys()) == [2,8]
print("Sanity Check Passed!")

Sanity Check Passed!


## ADD history to dataframe

control
control
control
control
repeated
                                        gameID  trialNum condition  \
686  0055-b8f4d6a3-ff78-4051-bd88-0fb01a2f5764         7  repeated   
691  0055-b8f4d6a3-ff78-4051-bd88-0fb01a2f5764        12  repeated   
695  0055-b8f4d6a3-ff78-4051-bd88-0fb01a2f5764        16  repeated   
698  0055-b8f4d6a3-ff78-4051-bd88-0fb01a2f5764        19  repeated   
701  0055-b8f4d6a3-ff78-4051-bd88-0fb01a2f5764        22  repeated   
707  0055-b8f4d6a3-ff78-4051-bd88-0fb01a2f5764        28  repeated   
710  0055-b8f4d6a3-ff78-4051-bd88-0fb01a2f5764        31  repeated   
718  0055-b8f4d6a3-ff78-4051-bd88-0fb01a2f5764        39  repeated   

         target category  repetition     phase generalization  drawDuration  \
686  waiting_00  waiting           0       pre        between         8.813   
691  waiting_00  waiting           1  repeated        between        10.062   
695  waiting_00  waiting           2  repeated        between         8.735   
698  waiting

In [30]:
M['parts'] = M.apply(lambda x: {"armrest":[], "legs": ["front-left leg", "front-right leg", "back-left leg", "back-right leg"]}, axis = 1)
M.to_csv('./graphical_conventions_group_data_run5_submitButton_with_TargetURL.csv')

## Decode the pngString to images

In [21]:
import base64
pngPath = os.path.join(path_to_stim, 'pngs')

if not os.path.exists(pngPath):
    os.mkdir(pngPath)
    print("Directory " , pngPath ,  " Created ")
else:    
    print("Directory " , pngPath ,  " already exists")
    
for index, row in M.iterrows():
    g = row['gameID']
    l = row['target']
    r = row['repetition']
    t = row['trialNum']
    c = row['condition']
    s = 'both'
    e = 'graphical-convention'
    this_fname = '{}_{}_{}_{}_{}_{}_{}'.format(g,l,r,t,c,s,e)
    
    fh = open(os.path.join(pngPath,this_fname), "wb")
    #png = row.png.encode()
    imgdata = base64.b64decode(row.png)
    fh.write(imgdata)
    fh.close()
    

Directory  /Users/yiyang/project/semantic_parts_v2/data/pngs  already exists


## Renders the svg to pngs

In [76]:
import rendering_helpers as srh

## extract sketch identifying info
gseries = M['gameID'].map(str)
nseries = M['trialNum'].map(str).apply(lambda x: x.zfill(2))
rseries = M['repetition'].map(str).apply(lambda x: x.zfill(2))
tseries = M['target'].map(str)

## build list of image filenames
fname_list = ['{}_{}_{}'.format(i,j,k) for (i,j,k) in zip(gseries,rseries,tseries)]

## convert svg string strings into svg string list
svg_string_list = [ast.literal_eval(i) for i in M.svgString.values]
## render out svg & convert to png
reload(srh)

<module 'rendering_helpers' from '/Users/yiyang/project/semantic_parts_v2/data/rendering_helpers.py'>

In [77]:
reallyRun = 0
i = 0
tot = len(M)
if reallyRun:
    for this_fname,this_svg in zip(fname_list,svg_string_list):    
        srh.render_svg(this_svg,base_dir=sketch_dir,out_fname= '{}.svg'.format(this_fname))    
        clear_output(wait=True)
        print(i, "out of: ", tot)
        i+=1
        
        
    ## get svg path list for rendered out svg
    svg_paths = srh.generate_svg_path_list(os.path.join(sketch_dir,'svg'))    
    
    ## convert all svg to png
    srh.svg_to_png(svg_paths,base_dir=sketch_dir)

convert /Users/yiyang/project/semantic_parts_v2/sketches/svg/6225-fa50e13d-b205-4a00-b12a-493b412627af_05_waiting_03.svg /Users/yiyang/project/semantic_parts_v2/sketches/png/6225-fa50e13d-b205-4a00-b12a-493b412627af_05_waiting_03.png


## Upload to AWS

In [81]:
import boto
bucket_name = 'graphical-conventions-sketches'
path_to_png = os.path.join(sketch_dir,'png')
runThis = 0
if runThis:
    conn = boto.connect_s3()
    b = conn.create_bucket(bucket_name) ### if bucket already exists, then get_bucket, else create_bucket
    for ind,im in enumerate(os.listdir(path_to_png)):
        if im[-3:]=='png':
            print (ind, im)
            k = b.new_key(im)
            k.set_contents_from_filename(os.path.join(path_to_png,im))
            k.set_acl('public-read')
            clear_output(wait=True)

## Add history to dataframe

In [ ]:
for i, group in M.groupby(['gameID','target']):
    print(group.iloc[0].condition)
    if group.iloc[0].condition == "control":
        continue
    print(group)
    break
